# Sistema de recomendación de anime

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd 
from sklearn.neighbors import NearestNeighbors

El objetivo es aplicar todo lo que hemos aprendido hasta el momento, especialmente sobre gradiente descendiente. Para esto desarrollaremos un sistema de recomendación usando factorización de matrices.

En esta entrega se darán todos elementos necesarios para desarrollar un sistema de recomendación de amines, el objetivo es que usted entienda como funciona y desarrolle una solución.

Si desea conocer mas sobre el dataset que usaremos puede revisar acá: https://www.kaggle.com/CooperUnion/anime-recommendations-database

1- Cargue las bases de datos de calificaciones, usuarios y animes. Puede usar para este objetivo la biblioteca pandas. Genere el conjunto de entrenamiento usando los usuarios cómo filas y los animes cómo columnas. Tenga en cuenta que para el algoritmo es necesario que los -1 sean cero, sin embargo, para la recomendación si debe tener en cuenta dichos valores.

In [2]:
df = pd.read_csv('anime.csv', delimiter=',', parse_dates=[6], header=0,index_col=False, squeeze=False)
d=np.array(df)
print(d)
uf = pd.read_csv('rating.csv', delimiter=',', parse_dates=[2], header=0,index_col=False, squeeze=False)
u=np.array(uf)
print(u)

[[32281 'Kimi no Na wa.' 'Drama, Romance, School, Supernatural' ... '1'
  9.37 '200630']
 [5114 'Fullmetal Alchemist: Brotherhood'
  'Action, Adventure, Drama, Fantasy, Magic, Military, Shounen' ... '64'
  9.26 '793665']
 [28977 'Gintama°'
  'Action, Comedy, Historical, Parody, Samurai, Sci-Fi, Shounen' ... '51'
  9.25 '114262']
 ...
 [5621 'Violence Gekiga David no Hoshi' 'Hentai' ... '4' 4.88 '219']
 [6133 'Violence Gekiga Shin David no Hoshi: Inma Densetsu' 'Hentai' ...
  '1' 4.98 '175']
 [26081 'Yasuji no Pornorama: Yacchimae!!' 'Hentai' ... '1' 5.46 '142']]
[[1 20 '-1']
 [1 24 '-1']
 [1 79 '-1']
 ...
 [73515 22145 '10']
 [73516 790 '9']
 [73516 8074 '9']]


2- Agregue la función donde se ejecuta el algoritmo de factorización de matrices. Puede basarse en el algoritmo compartido en las dispositivas. No debe de gastar todas las interaciones. 

In [3]:
def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in xrange(steps):
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - numpy.dot(P[i,:],Q[:,j])
                    for k in xrange(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = numpy.dot(P,Q)
        e = 0
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - numpy.dot(P[i,:],Q[:,j]), 2)
                    for k in xrange(K):
                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        if e < 0.001:
            break
    return P, Q.T

3- Programe una función para generar $\hat{R}$. Recuerde que para generar $\hat{R}$ debe realizar los siguiente pasos:

    - Generar R usando el algoritmo de vecinos más cercanos.
    - Defina K.
    - Defina P y Q.
    - Generar R prima.


In [ ]:
nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(u)
distances, indices = nbrs.kneighbors(u)
print(indices)

4- Realice las siguiente recomendaciones (animes que no ha visto con calificación 10):

    1- Usuario de id 51, películas.
    2- Usuario de id 510, animes con menos de 31 episodios.
    3- Usuario de id 17000, animes con rankins mayores o iguales a 9.
    4- Usuario de id 27005, animes de comedia y aventura.
    5- Usuario de id 37502, TV.
    6- Usuario de id 57502, películas y OVAS.
    7- Usuario de id 67501.

Cada recomendación debe tener como respuesta, número de iteraciones, número de recomendaciones y nombre de los animes,